## Random Forest

Random Forest is one of my favorite methods because it is pretty easy to use and returns pretty good results. The data has already been cleaned up so I can get straight to the modeling from here. I'll also be creating a table of results I get from the algorithm. The first table will be based on performance and accuracy I'll be manipulating the data and my settings to get the best result.

Here is some data from earlier runs that I did earlier in the week. I don't have enough RAM on my computer to run a proper Random Forest on the full data set so I tried changing some of the settings to see what type of results I would get, so far it things don't look so great.

| Score               | Depth | Trees |
|---------------------|-------|-------|
| 0.24406610565014025 | 7     | 250   |
| 0.243709633517      | 7     | 500   |
| 0.244111661194      | 7     | 750   |
| 0.243918050132      | 7     | 1000  |
| 0.29118306609312239 | 15    | 250   |

In [1]:
import pandas as pd
import numpy as np
Y = pd.read_csv('./data/Y.csv', index_col = 0, header=None, names=['Crime'])
X = pd.read_csv('./data/X.csv', index_col=0)

In [8]:
rows = np.random.choice(X.index.values, X.shape[0]*.30)
a = X.ix[rows]
b = Y['Crime'].ix[rows]

In [6]:
import sklearn.ensemble as sk
n = 25
rfc = sk.RandomForestClassifier(n_jobs=-1, n_estimators = n, oob_score = True, bootstrap = True)

In [9]:
%time rfc.fit(a,b)
rfc.oob_score_

Wall time: 13.5 s


C:\Users\2015\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:379: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


0.40689940549856879

In [ ]:
# 1% 80mb 453ms 18.34% 25
# 1% 750mb 3.52ms 22.79% 250
# 1% 2500
# 1% 2800mb 13.4s 24.27% 1000
# 10% 700mb 4.07s 27.91% 25
# 30% 1700mb 13.5s 40.68% 25

